# extract type 0 data only
## and removed answers with magnitude more than 3

In [3]:
import json
def extract_id(start_token='0', filepath=r'alldata.json'):
    start = start_token
    texts = []
    with open(filepath,"r",encoding='utf-8') as f:
        data=json.load(f)
    for key,item in data.items():
        questions=item['question']
        for q in questions:
            q_temp,a=q.split('     ')
            idx,q = q_temp.split("=")
            idx_=idx.split(" ")[-2]
            if idx_.startswith(start):
                text={}
                text['q_id']=idx_
                text['question']=q.strip()
                text['answer']=a
                text['doc_id']=key.strip()
                texts.append(text)
    return texts

In [4]:
#extract type 0 qa pairs
fp = '/Users/rickzhai/Desktop/semeval data/training data/datas_1120.json'
t = extract_id(start_token='0', filepath=fp)
write = open("qa_0_all.json", "w", encoding="utf-8")
json.dump(t, write, ensure_ascii=False, indent=4)
write.close()

# create df as type_0 dataset

In [5]:
import pandas as pd
qa = r"/Users/rickzhai/Documents/GitHub/ITNLP_Semeval2022_Task6/number_reasoning/qa_0_all.json"
recipe = r"/Users/rickzhai/Documents/GitHub/ITNLP_Semeval2022_Task6/number_reasoning/padded_text_all.json"
with open(qa,"r",encoding='utf-8') as f:
    qa_0=json.load(f)
with open(recipe,"r",encoding='utf-8') as f:
    c=json.load(f)
data = {}
data['question'] = []
data['answer'] = []
data['context'] = []
for _ in qa_0:
    if _.get('question'):
        data['question'].append(_.get('question'))
        data['answer'].append(int(_.get('answer')))
        docid = _.get('doc_id')
        the_doc = next((sub for sub in c if sub['doc_id'] == docid), None)
        context = ''.join(the_doc['text'])
        data['context'].append(context)


In [6]:
import pandas as pd

# write to csv for train

In [7]:
df = pd.DataFrame(data)
df.to_csv('qa_0_all.csv', encoding='utf-8')

In [25]:
print(type(str(df.iloc[1821].answer)))

<class 'str'>


In [12]:
len(df)

3306

In [62]:
#pip install sentencepiece

     |████████████████████████████████| 1.1 MB 8.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [18]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the BERT tokenizer.
print('Loading ALBERT tokenizer...')
l='allenai/longformer-large-4096-finetuned-triviaqa'
b='google/bigbird-roberta-large'
tokenizer = AutoTokenizer.from_pretrained(l, do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained(l, num_labels=3)


Loading ALBERT tokenizer...


Some weights of the model checkpoint at allenai/longformer-large-4096-finetuned-triviaqa were not used when initializing LongformerForSequenceClassification: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-large-4096-finetuned-triviaqa and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You sho

In [21]:
import torch
inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels, return_dict = False)


In [22]:
len(outputs)

2

In [24]:
params = list(model.named_parameters())
print('==== Embedding Layer ====\n')

for p in params:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

==== Embedding Layer ====

longformer.embeddings.word_embeddings.weight            (50265, 1024)
longformer.embeddings.position_embeddings.weight        (4098, 1024)
longformer.embeddings.token_type_embeddings.weight         (1, 1024)
longformer.embeddings.LayerNorm.weight                       (1024,)
longformer.embeddings.LayerNorm.bias                         (1024,)
longformer.encoder.layer.0.attention.self.query.weight  (1024, 1024)
longformer.encoder.layer.0.attention.self.query.bias         (1024,)
longformer.encoder.layer.0.attention.self.key.weight    (1024, 1024)
longformer.encoder.layer.0.attention.self.key.bias           (1024,)
longformer.encoder.layer.0.attention.self.value.weight  (1024, 1024)
longformer.encoder.layer.0.attention.self.value.bias         (1024,)
longformer.encoder.layer.0.attention.self.query_global.weight (1024, 1024)
longformer.encoder.layer.0.attention.self.query_global.bias      (1024,)
longformer.encoder.layer.0.attention.self.key_global.weight (1024

In [25]:
import pandas as pd
df = pd.read_csv('./qa_0.csv')

max_len = 0

# For every sentence...
for sent in df.context.values:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)

    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  1244


In [23]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [26]:
import torch
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []
sentences = df.context.values
qs = df.question.values
# For every sentence...
for sent in zip(qs,sentences):
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent[0],
                        sent[1],                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                                  # Pad & truncate all sentences.
                                   # Pad & truncate all sentences.
                        max_length = 1024,
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
#labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
#print('Original: ', sentences[0])
#print(input_ids[0][:273])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/Users/rickzhai/opt/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2212: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [27]:
print(input_ids[0][:660])

tensor([    0,  6179,   171,  2163,   473,    24,   185,     7,   609,     5,
        37570,  4884,   116,     2,     2,  8267,   849,   898,  4832, 16639,
         8942,     6,  3944,  4832,  7023,   849,     5, 34803,   849,  2731,
          506,  4832, 34803,   849,  1437,   849,   233,  4832,   847,   849,
           88,  3041,  2816,  1872,   849,  2731,   506,  4832, 16639,  8942,
          849,     8,     5, 10114,   849,  2731,   506,  4832, 34803,   849,
         1437,   849,   233,  4832,   847,   849,    88, 10970,   111,  1836,
         3745,   849,  2731,   506,  4832, 16639,  8942,   849, 25606,   847,
          849,  3944,  4832,  7023,   849, 28488,   849,  2731,   506,  4832,
        28488,   849,  1437,   849,   233,  4832,   847,   849,     8,   992,
         3964, 33764,   849,  2731,   506,  4832,   992,  3964, 33764,   849,
         1437,   849,   233,  4832,   847,   849,    88, 35788,   849,  2731,
          506,  4832, 16639,  8942,   849,   479,  2241,  4467, 

In [34]:
labels = df.answer.astype(int).values 
l1 = 0
l2 = 0
l3 = 0
for _ in labels:
    if _ == 1:
        l1+=1
    if _ == 2:
        l2+=1
    if _ == 3:
        l3+=1
print(l1,l2,l3)
print(len(labels))

2395 482 178
3055


# preprocess tags 
## make them pretty

In [9]:
import json
import re
regex = re.compile('[^a-zA-Z]') # to remove .1.2.3 and _ in hidden tags

def useful_tag_8(tags: str):
    tags = tags.lower()
    if "|" in tags:
        tags = tags.split("|")
        ans = []
        for tag in tags:
            if tag:
                name, tag = tag.split("=")
                taglist = tag.split(":")
                regextaglist = [regex.sub(' ', _) for _ in taglist]

                alltag = " ".join(regextaglist)
                ans.append(name + " : " + alltag)
                # replace multiple blankspace with one
                newans = [' '.join(_.split()) for _ in ans] 
        if len(newans) != 0:
            return newans
        else:
            return "_"

    else:
        ans = []
        if tags != '_':

            name, tag = tags.split("=")
            taglist = tag.split(":")
            regextaglist = [regex.sub(' ', _) for _ in taglist]

            alltag = " ".join(regextaglist)
            ans.append(name + " : " + alltag)
            newans = [' '.join(_.split()) for _ in ans]
            if len(newans) != 0:
                return newans
        else:
            return "_"

def part_to_text(partlist:list,text:str): # change part number to the word
    text_list = text.split()
    for i, _ in enumerate(partlist):
        if _ != '_':
            partlist[i] = text_list[int(_)-1]
    return partlist

# extract text & tag columns.json

In [13]:
import re
regex = re.compile('[^a-zA-Z]') # to remove .1.2.3 in tags
    
with open(r"/Users/rickzhai/Documents/GitHub/ITNLP_Semeval2022_Task6/crl_srl.csv", 'r', encoding='utf-8') as f:
    all_data = f.readlines()
all_text_tag = []  # store all text tag columns
i = 1
while i < 219124:
    doc_id = all_data[i - 1].split("=")[1].strip("\n")  
    text_tags = []  
    while i < 219124 and 'newdoc id' not in all_data[i]:
        item = all_data[i]
        if i+1 < 219124:
            item_next = all_data[i + 1] 
        
        if 'text =' in item:
            item_pre = all_data[i - 1]
            if 'ingredients' in item_pre:
                i += 1
                pass
            else:
                i += 1
                temp_text_tags = {}
                texts, tagcols = "", {}

                tagcols['word'] = []
                tagcols['lemma'] = []
                tagcols['pos'] = []
                tagcols['entity'] = []
                tagcols['part'] = []
                tagcols['result'] = []
                tagcols['hidden'] = []
                tagcols['coref'] = []
                tagcols['prdct'] = []
                tagcols['arg1'] = []
                tagcols['arg2'] = []
                tagcols['arg3'] = []
                tagcols['arg4'] = []
                tagcols['arg5'] = []
                tagcols['arg6'] = []
                tagcols['arg7'] = []
                tagcols['arg8'] = []
                tagcols['arg9'] = []
                tagcols['arg10'] = []
                

                while i < 219125 and all_data[i] and "sent_id" not in all_data[i] and 'newdoc id' not in all_data[i]: 
                    if "newpar id" in all_data[i]:
                        i += 1
                        continue
                    item = all_data[i]

                    tags = item.strip().split("\t")
                    if len(tags) != 1:

                        tagcols['word'].append(tags[1])
                        tagcols['lemma'].append(tags[2])
                        tagcols['pos'].append(tags[3])
                        tagcols['entity'].append(tags[4])
                        tagcols['part'].append(tags[5])
                        tagcols['result'].append(tags[6])
                        tagcols['hidden'].append(useful_tag_8(tags[7])) # list of hidden tags ['shadow : water', 'habitat : pot']
                        if tags[8] != '_':
                            tagcols['coref'].append(regex.sub(' ',tags[8]).strip())
                        else:
                            tagcols['coref'].append(tags[8])
                        tagcols['prdct'].append(tags[9])
                        tagcols['arg1'].append(tags[10])
                        tagcols['arg2'].append(tags[11])
                        tagcols['arg3'].append(tags[12])
                        tagcols['arg4'].append(tags[13])
                        tagcols['arg5'].append(tags[14])
                        tagcols['arg6'].append(tags[15])
                        tagcols['arg7'].append(tags[16])
                        tagcols['arg8'].append(tags[17])
                        tagcols['arg9'].append(tags[18])
                        tagcols['arg10'].append(tags[19])
                        
                        texts += (" " + tags[1].lower())

                    i += 1
                temp_text_tags["text"] = texts.strip()
                tagcols['part'] = part_to_text(tagcols['part'],temp_text_tags["text"]) # from coref number to the word
                temp_text_tags["tagcols"] = tagcols
                text_tags.append(temp_text_tags)
        else:
            i += 1
    i += 1

    doc_dict = {}
    doc_dict['doc_id'] = doc_id
    doc_dict['text_tags'] = text_tags
    all_text_tag.append(doc_dict)
fw = open("all_text_tag.json", "w", encoding="utf-8")
json.dump(all_text_tag, fw, ensure_ascii=False, indent=4)
fw.close()

In [10]:
import itertools
def mergeTextTag(columns:tuple, jsonFile):
    
    '''
    merge text and tags from json file in the form:    
    text text # TAG = tag # # TAG = tag # text text .
    
    parameters: 
        columns: tuple of str, where the tags are from, tags can from multiple columns
        json: str, the input json file address
    
    input: 
        column idx in tuple
        json data file
        
    '''
    file = json.load(jsonFile)
    paddeddata = []
    
    for each_doc in file:
        paddeddoc = {}
        paddeddoc['text']=[]
        paddeddoc['doc_id']=each_doc['doc_id'].strip()
        
        texttag_list = each_doc['text_tags'] # list
        
        # allign each word with target columns tags of it
        for _ in texttag_list: 
            tags = _['tagcols'] 
            text_list = _['text'].split(' ') # the sentence list
            
            # combine cols of tags together in list.
            tags_to_merge = [list(t) for t in zip(tags[c] for c in columns)] 
            unested = [list(itertools.chain(*sub)) for sub in tags_to_merge]
            
            # unzip the hidden tag list
            for l in unested:
                for i, v in enumerate(l):
                    if type(v) == list:
                        l[i] = ', '.join(v)
            
            # add '#' at the start and end of tags 
            newtag = []
            for idx, _ in enumerate(zip(*unested)): # add tag name and '#' for not meaningful tags
                if not all( l == '_' for l in _ ):
                    hashpadlist =['# ' + columns[i] + ' : ' + t + ' # ' \
                                for i, t in enumerate(_) if t != '_']
                    hashpad = ' '.join(hashpadlist)
                    newtag.append(hashpad)
                else:
                    newtag.append('')
            
            paddedtext = ''
            # and merge with the text list 
            
            for text, tag in zip(text_list, newtag):
                paddedtext += text + ' ' + tag
            pt = paddedtext.replace('hidden : ','')
            # append padded text to the list
            paddeddoc['text'].append(pt)
        paddeddata.append(paddeddoc)
    return paddeddata


# merge text and tag
## in the form: ' text text # TAG = tag # # TAG = tag # text text '

In [14]:

jsonpath = r'/Users/rickzhai/Documents/GitHub/ITNLP_Semeval2022_Task6/number_reasoning/all_text_tag.json'
f = open(jsonpath, 'r', encoding='utf-8')
padded = mergeTextTag(columns=('coref','part','hidden'), jsonFile=f)
w = open("padded_text_all.json", "w", encoding="utf-8")
json.dump(padded, w, ensure_ascii=False, indent=4)
w.close()
